In [4]:
import pvlib
import pandas as pd

# set location parameters
lat = 33.084
lon = -96.699
elevation = 200 # in meters

# load TMY data
tmy_data, _ = pvlib.iotools.read_tmy3('USA_TX_McKinney-Muni.AP.722630_TMY3.epw')

# create location object
location = pvlib.location.Location(lat=lat, lon=lon, altitude=elevation, tz='Etc/GMT+6')

# calculate solar position
solar_position = location.get_solarposition(tmy_data.index)

# calculate clear sky GHI and DNI
clear_sky = location.get_clearsky(tmy_data.index)
dni_extra = pvlib.irradiance.get_extra_radiation(tmy_data.index)
clear_sky['dni'] = pvlib.irradiance.dirindex(clear_sky['ghi'], solar_position['apparent_zenith'], tmy_data.index, method='simple') * dni_extra
clear_sky['dhi'] = clear_sky['ghi'] - clear_sky['dni'] * solar_position['apparent_zenith'].apply(lambda x: max(x, 0)).apply(lambda x: min(x, pvlib.irradiance.atmosphere.get_relative_airmass(x)))

# calculate surface tilt and azimuth
surface_azimuth = 180  # facing south
surface_tilt = pvlib.irradiance.getSlope(solar_position['apparent_zenith'], solar_position['azimuth'], surface_azimuth)

# calculate irradiance components on the tracking surface
irradiance = pvlib.irradiance.get_total_irradiance(surface_tilt, surface_azimuth, solar_position['apparent_zenith'], solar_position['azimuth'], clear_sky['dni'], clear_sky['ghi'], clear_sky['dhi'], dni_extra=dni_extra, model='haydavies')

# calculate AC power output
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
module = sandia_modules['SunPower_SPR_345_WHT']
inverter = pvlib.pvsystem.retrieve_sam('CECInverter', 'SMA_America__SB7200TL_US_72__208V_')
system = pvlib.pvsystem.PVSystem(surface_tilt=surface_tilt, surface_azimuth=surface_azimuth, module_parameters=module, inverter_parameters=inverter)
ac_power = system.ac_power(irradiance['dni'], irradiance['ghi'], irradiance['dhi'], location=location)

# calculate LCOE
discount_rate = 0.07 # in decimal
plant_cost = 2000 # in $/kW DC
federal_tax_rate = 0.3 # in decimal
state_tax_rate = 0.05 # in decimal
operation_maintenance_cost = 25 # in $/kW DC/year
lifetime = 25 # in years
energy = ac_power.sum() / 1000 # in MWh
capacity_factor = energy / (8760 * system.module_parameters['Impo'] * system.module_parameters['Vmpo'] * system.module_parameters['Area'])  # assuming system granularity of 1 module
capacity = 1 / capacity_factor
capital_recovery_factor = discount_rate * (1 + discount_rate) ** lifetime / ((1 + discount_rate) ** lifetime - 1)
total_annual_cost = ((plant_cost + operation_maintenance_cost) / capacity * capital_recovery_factor + operation_maintenance_cost) * (1 - federal_tax_rate - state_tax_rate)
lcoe = total_annual_cost / energy

print('LCOE = {:.2f} $/MWh'.format(lcoe))


ModuleNotFoundError: No module named 'pvlib'